In [106]:
# Milestone 1: Data Ingestion and Preparation
# Installing the required libraries
!pip install pyspark pandas

In [108]:
# Importing necessary libraries
from pyspark.sql import SparkSession
import pandas as pd

# Initializing spark session
spark = SparkSession.builder.appName("MovieLens").getOrCreate()

# Defining the column names
data_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
item_cols = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
             'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance ',
             'Sci-Fi', 'Thriller', 'War', 'Western']
user_cols = ['user_id', 'age', 'gender', 'occupation', 'zip_code']

In [109]:
# Loading the data files using pandas
data = pd.read_csv('/content/u.data', sep='\t', names=data_cols, encoding='latin-1')
item = pd.read_csv('/content/u.item', sep='|', names=item_cols, encoding='latin-1')
users = pd.read_csv('/content/u.user', sep='|', names=user_cols, encoding='latin-1')

In [110]:
# Displaying the first few rows of each dataframe
# data.head()
data.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [111]:
# item.head()
item.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [112]:
# users.head()
users.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [113]:
# Converting pandas dataframes to spark dataframes
data_df = spark.createDataFrame(data)
item_df = spark.createDataFrame(item)
users_df = spark.createDataFrame(users)

In [114]:
# Milestone 2: Exploratory Data Analysis with SQL
# Registering dataframes as temporary views
data_df.createOrReplaceTempView("data")
item_df.createOrReplaceTempView("item")
users_df.createOrReplaceTempView("users")

In [115]:
# Identifying the most rated movies
print("\nMost rated movies:")
spark.sql("""
SELECT movie_title, COUNT(*) as rating_count
FROM item JOIN data ON item.movie_id = data.movie_id
GROUP BY movie_title
ORDER BY rating_count DESC
""").show(10)



Most rated movies:
+--------------------+------------+
|         movie_title|rating_count|
+--------------------+------------+
|    Star Wars (1977)|         583|
|      Contact (1997)|         509|
|        Fargo (1996)|         508|
|Return of the Jed...|         507|
|    Liar Liar (1997)|         485|
|English Patient, ...|         481|
|       Scream (1996)|         478|
|    Toy Story (1995)|         452|
|Air Force One (1997)|         431|
|Independence Day ...|         429|
+--------------------+------------+
only showing top 10 rows



In [116]:
# Counting the number of ratings each user has given
print("\nUsers and the count of ratings users've given:")
spark.sql("""
SELECT user_id, COUNT(*) as rating_count
FROM data
GROUP BY user_id
ORDER BY rating_count DESC
""").show(10)



Users and the count of ratings users've given:
+-------+------------+
|user_id|rating_count|
+-------+------------+
|    405|         737|
|    655|         685|
|     13|         636|
|    450|         540|
|    276|         518|
|    416|         493|
|    537|         490|
|    303|         484|
|    234|         480|
|    393|         448|
+-------+------------+
only showing top 10 rows



In [117]:
# Identifying movie IDs with at least one user rating
print("\nMovie IDs with at least one user rating:")
spark.sql("""
SELECT DISTINCT movie_id
FROM data
""").show(10)


Movie IDs with at least one user rating:
+--------+
|movie_id|
+--------+
|     474|
|      29|
|      26|
|     964|
|      65|
|     191|
|    1224|
|     558|
|    1010|
|     418|
+--------+
only showing top 10 rows



In [118]:
# Listing users who have rated at least one movie
print("\nUsers who have rated at least one movie:")
spark.sql("""
SELECT DISTINCT user_id
FROM data
""").show(10)



Users who have rated at least one movie:
+-------+
|user_id|
+-------+
|     26|
|     29|
|    474|
|    191|
|     65|
|    418|
|    541|
|    558|
|    293|
|    222|
+-------+
only showing top 10 rows



In [119]:
# Milestone 3: User Ratings Analysis
# Average, Min, and Max ratings for each movie
from pyspark.sql.functions import avg, min, max
print("\nAverage, Min, and Max ratings for each movie:")
movie_stats = data_df.groupBy("movie_id").agg(
    avg("rating").alias("avg_rating"),
    min("rating").alias("min_rating"),
    max("rating").alias("max_rating")
)
movie_stats.show(10)



Average, Min, and Max ratings for each movie:
+--------+------------------+----------+----------+
|movie_id|        avg_rating|min_rating|max_rating|
+--------+------------------+----------+----------+
|     474| 4.252577319587629|         2|         5|
|      29|2.6666666666666665|         1|         5|
|      26| 3.452054794520548|         1|         5|
|     964|3.3333333333333335|         1|         5|
|      65|3.5391304347826087|         1|         5|
|     191| 4.163043478260869|         1|         5|
|    1224|2.6666666666666665|         1|         4|
|     558|3.6714285714285713|         1|         5|
|    1010|              3.25|         1|         5|
|     418|3.5813953488372094|         1|         5|
+--------+------------------+----------+----------+
only showing top 10 rows



In [120]:
# Average, Min, and Max ratings for each user
print("\nAverage, Min, and Max ratings for each user:")
user_stats = data_df.groupBy("user_id").agg(
    avg("rating").alias("avg_rating"),
    min("rating").alias("min_rating"),
    max("rating").alias("max_rating")
)
user_stats.show(10)



Average, Min, and Max ratings for each user:
+-------+------------------+----------+----------+
|user_id|        avg_rating|min_rating|max_rating|
+-------+------------------+----------+----------+
|     26|  2.94392523364486|         1|         5|
|     29|3.6470588235294117|         2|         5|
|    474|  4.08256880733945|         1|         5|
|    191|3.6296296296296298|         2|         5|
|     65|            3.9375|         1|         5|
|    418|               2.9|         1|         5|
|    541|3.6240601503759398|         1|         5|
|    558|               4.2|         2|         5|
|    293|3.0309278350515463|         1|         5|
|    222| 3.049095607235142|         1|         5|
+-------+------------------+----------+----------+
only showing top 10 rows



In [122]:
# Counting of users who are artists
artist_count = users_df.filter(users_df.occupation == "artist").count()
print(f"Number of users who are artists: {artist_count}")


Number of users who are artists: 28


In [123]:
# Counting of users who are artists and at least 25 years old
artists_25_plus = users_df.filter((users_df.occupation == "artist") & (users_df.age >= 25)).count()
print(f"Number of users who are artists and at least 25 years old: {artists_25_plus}")


Number of users who are artists and at least 25 years old: 22


In [124]:
# Milestone 4: PySpark DataFrame API Implementation
# Listing all movies along with the number of ratings
print("\nAll Movies along with the number of ratings:")
movies_with_ratings_count = data_df.groupBy("movie_id").count()
movies_with_ratings_count.show(10)



All Movies along with the number of ratings:
+--------+-----+
|movie_id|count|
+--------+-----+
|     474|  194|
|      29|  114|
|      26|   73|
|     964|    9|
|      65|  115|
|     191|  276|
|    1224|   12|
|     558|   70|
|    1010|   44|
|     418|  129|
+--------+-----+
only showing top 10 rows



In [125]:
# Displaying users and the count of ratings users've given for movies
print("\nUsers and the count of ratings users've given for movies:")
user_ratings_count = data_df.groupBy("user_id").count()
user_ratings_count.show(10)



Users and the count of ratings users've given for movies:
+-------+-----+
|user_id|count|
+-------+-----+
|     26|  107|
|     29|   34|
|    474|  327|
|    191|   27|
|     65|   80|
|    418|   20|
|    541|  133|
|    558|   20|
|    293|  388|
|    222|  387|
+-------+-----+
only showing top 10 rows



In [126]:
# Identify movie IDs with at least one user rating
print("\nMovie IDs with at least one user rating:")
movie_ids_with_ratings = data_df.select("movie_id").distinct()
movie_ids_with_ratings.show(10)



Movie IDs with at least one user rating:
+--------+
|movie_id|
+--------+
|     474|
|      29|
|      26|
|     964|
|      65|
|     191|
|    1224|
|     558|
|    1010|
|     418|
+--------+
only showing top 10 rows



In [127]:
# Listing users who have rated at least one movie
print("\nUsers who have rated at least one movie:")
users_with_ratings = data_df.select("user_id").distinct()
users_with_ratings.show(10)


Users who have rated at least one movie:
+-------+
|user_id|
+-------+
|     26|
|     29|
|    474|
|    191|
|     65|
|    418|
|    541|
|    558|
|    293|
|    222|
+-------+
only showing top 10 rows



In [128]:
# Providing lists of users and movies with their Maximum, Minimum, and Average ratings
# Users
print("\nUser ratings analysis (Max, Min, Avg):")
user_stats = data_df.groupBy("user_id").agg(
    avg("rating").alias("avg_rating"),
    min("rating").alias("min_rating"),
    max("rating").alias("max_rating")
)
user_stats.show(10)


User ratings analysis (Max, Min, Avg):
+-------+------------------+----------+----------+
|user_id|        avg_rating|min_rating|max_rating|
+-------+------------------+----------+----------+
|     26|  2.94392523364486|         1|         5|
|     29|3.6470588235294117|         2|         5|
|    474|  4.08256880733945|         1|         5|
|    191|3.6296296296296298|         2|         5|
|     65|            3.9375|         1|         5|
|    418|               2.9|         1|         5|
|    541|3.6240601503759398|         1|         5|
|    558|               4.2|         2|         5|
|    293|3.0309278350515463|         1|         5|
|    222| 3.049095607235142|         1|         5|
+-------+------------------+----------+----------+
only showing top 10 rows



In [129]:
# For Movies
print("\nMovie ratings analysis (Max, Min, Avg):")
movie_stats = data_df.groupBy("movie_id").agg(
    avg("rating").alias("avg_rating"),
    min("rating").alias("min_rating"),
    max("rating").alias("max_rating")
)
movie_stats.show(10)


Movie ratings analysis (Max, Min, Avg):
+--------+------------------+----------+----------+
|movie_id|        avg_rating|min_rating|max_rating|
+--------+------------------+----------+----------+
|     474| 4.252577319587629|         2|         5|
|      29|2.6666666666666665|         1|         5|
|      26| 3.452054794520548|         1|         5|
|     964|3.3333333333333335|         1|         5|
|      65|3.5391304347826087|         1|         5|
|     191| 4.163043478260869|         1|         5|
|    1224|2.6666666666666665|         1|         4|
|     558|3.6714285714285713|         1|         5|
|    1010|              3.25|         1|         5|
|     418|3.5813953488372094|         1|         5|
+--------+------------------+----------+----------+
only showing top 10 rows



In [130]:
# Use case descriptions of the MovieLens dataset
print("Uses of the MovieLens Dataset are as follows:\n")
print("Determine which movies have received the highest number of ratings.")
print("Discover movies that have the highest average ratings from users.")
print("Predict which movies are the most controversial among different age groups by analyzing the variance in ratings.")
print("Identify movies on which men and women most disagree, providing insights into gender-based preferences.")
print("Find out which occupations are most active in rating movies and their preferences.")
print("Develop a recommender system to suggest movies to users based on their rating history and preferences.")
print("Calculate the average, maximum, and minimum ratings of movies to understand their overall reception.")
print("Determine which movies are suitable for different age groups based on the ratings and reviews.")

Uses of the MovieLens Dataset are as follows:

Determine which movies have received the highest number of ratings.
Discover movies that have the highest average ratings from users.
Predict which movies are the most controversial among different age groups by analyzing the variance in ratings.
Identify movies on which men and women most disagree, providing insights into gender-based preferences.
Find out which occupations are most active in rating movies and their preferences.
Develop a recommender system to suggest movies to users based on their rating history and preferences.
Calculate the average, maximum, and minimum ratings of movies to understand their overall reception.
Determine which movies are suitable for different age groups based on the ratings and reviews.
